# <center> Feature Engineering </center>
<center> En este Jupyter Notebook vamos a preparar los conjuntos de datos necesarios para las funciones. </center>

## Importación de librerias

In [3]:
import pandas as pd
import numpy as np
import Utilities as u
import json
import ast 
import requests

from bs4 import BeautifulSoup
import nltk

nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Lectura de los datasets

In [4]:
reviews = pd.read_csv('./dataset/Modificados/reviews_sentiment_analysis.csv')
items = pd.read_csv('./dataset/Modificados/items.csv')
users = pd.read_csv('./dataset/Modificados/users.csv')
steam_games = pd.read_csv('./dataset/Modificados/steam_games.csv')

# Sentiment Analysis

Antes de preparar los conjuntos de datos, crearemos una nueva columna en el dataset **reviews** aplicando análisis de sentimiento con 'Natural Language Processing', el cual calisificara las resenias como:
* 0 : Malo
* 1 : Neutral
* 2 : Positivo

De no ser posible la clasificación por ausencia de la resenia, se tomara como Neutral.<br>
Esto facilitara el trabajo de los modelos de Machine Learning y el Análisis de datos.

In [120]:
reviews.head()

,user_id,item_id,posted,last_edited,funny,recommend,review,helpful
0,76561197970982479,1250,2011-11-05,NaN,NaN,True,Simple yet with great replayability. In my opi...,No ratings yet
1,76561197970982479,22200,2011-07-15,NaN,NaN,True,It's unique and worth a playthrough.,No ratings yet
2,76561197970982479,43110,2011-04-21,NaN,NaN,True,Great atmosphere. The gunplay can be a bit chu...,No ratings yet
3,js41637,251610,2014-06-24,NaN,NaN,True,I know what you think when you see this title ...,15 of 20 people (75%) found this review helpful
4,js41637,227300,2013-09-08,NaN,NaN,True,For a simple (it's actually not all that simpl...,0 of 1 people (0%) found this review helpful


Modificamos el tipo de datos de la columna *'review'* del dataset, para poder realizar el análisis.

In [37]:
reviews['review'] = reviews['review'].astype(str)

Creamos una función para poder analizar cada fila de la columna *'review'*, utilizando la libreria **NLTK**

In [127]:
def sentiment_analysis(text):
    '''
    Esta función utiliza la libreria NLTK para realizar un análisis de sentimiento de un texto
    y devuelve un valor númerico que categoriza cada texto en:
        0: NEGATIVO
        2: POSITIVO
        1: NEUTRAL
    

    Parameters:
        text (str): El texto que se va a analizar.

    Returns:
        int: Un valor numérico que representa el sentimiento del texto:
             - 0: Negativo.
             - 1: Neutral
             - 2: Positivo
    '''
    
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)

    if sentiment['compound'] >= 0.05:
        return 2
    elif sentiment['compound'] <= -0.05:
        return 0
    else:
        return 1

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Aplicamos esa función a cada registro de la columna *'review'*

In [128]:
reviews['sentiment_analysis'] = reviews['review'].apply(sentiment_analysis)

Esta nueva columna reemplazara a la columna de *'review'*,ya que nos facilitara el análisis. Asi que procederemos a eliminar la columna **review**

In [132]:
reviews.drop(columns ='review',inplace = True)

Procedemos a guardar el nuevo dataset en un archivo CSV

In [134]:
reviews.to_csv('./dataset/Modificados/reviews_sentiment_analysis.csv', encoding='utf-8',index=False)

In [429]:
reviews = pd.read_csv('./dataset/Modificados/reviews_sentiment_analysis.csv')

# Función ***userdata***

En esta función, toma como parámetro un *'user_id'* y se nos pide devolver la **cantidad de dinero gastado** por el usuario, el **porcentaje de recomendación** y la **cantidad de items**.

Para esta función vamos a necesitar:
* user_id : ID del usuario.
* total_spent: Cantidad de dinero gastado.
* items_count: Cantidad de items.
* recommend_total: Cantidad total de recomendaciones.
* percent_recommend: Porcentaje de recomendacion.

Sacamos las columnas necesarias

In [7]:
items_2 = items[['user_id','item_id']]
users_2 = users[['user_id','items_count']] 
steam_2 = steam_games[['id','price']]
reviews_2 = reviews[['user_id','recommend']]

Primero vamos a sacar la cantidad de dinero gastado por usuario.

In [8]:
items_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10814961 entries, 0 to 10814960
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   item_id  int64 
dtypes: int64(1), object(1)
memory usage: 165.0+ MB


In [9]:
steam_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32132 entries, 0 to 32131
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32132 non-null  float64
 1   price   32132 non-null  object 
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


Cambiamos el tipo de datos de la columna *'id'* para poder unir ambos DataFrames.

In [10]:
steam_2['id'] = steam_2['id'].astype(int)

Unimos los DataFrames.

In [11]:
df_function1 = items_2.merge(steam_2, left_on = 'item_id', right_on = 'id', how = 'left' )

In [12]:
df_function1.head()

,user_id,item_id,id,price
0,76561197970982479,10,10.0,9.99
1,76561197970982479,20,20.0,4.99
2,76561197970982479,30,30.0,4.99
3,76561197970982479,40,40.0,4.99
4,76561197970982479,50,50.0,4.99


In [158]:
df_function1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10814961 entries, 0 to 10814960
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  object 
 1   item_id  int64  
 2   id       float64
 3   price    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 330.0+ MB


In [13]:
df_function1.duplicated().sum()

5720879

Vemos que hay varios duplicados, asi que los eliminamos.

In [14]:
df_function1.drop_duplicates(inplace= True)

Eliminamos la columna *'id'* ya que tenemos la misma informacion en la columna *'items_id'*

In [15]:
df_function1.drop(columns= 'id',inplace= True)

Verificamos los valores nulos.

In [16]:
u.valores_nulos(df_function1)

La columna user_id tiene un 0.0% de valores nulos.
La columna item_id tiene un 0.0% de valores nulos.
La columna price tiene un 16.7% de valores nulos.


Vemos que hay un 16,7% de registros vacios en la columna **price**, y asumimos que si no tienen precio es porque son gratuitos, asi que lo modificamos por '0'.

In [17]:
df_function1['price'].fillna(0.0, inplace=True)

Veamos un poco los registros de la columna **price**

In [18]:
df_function1.price.unique()

array(['9.99', '4.99', '19.99', 0.0, '6.99', '--', '7.99', '2.99',
       '14.99', '0.0', '54.99', '29.99', '39.99', '24.99', '1.99',
       '11.99', '0.49', '0.98', '0.99', '59.99', '3.99', '2.49', '74.76',
       '5.99', '32.99', '12.99', '49.99', '13.98', '8.99', '13.99',
       '34.99', '12.89', '20.0', '14.95', '61.99', '15.99', '18.99',
       '17.99', '160.91', '59.95', '44.99', '3.49', '1.87', '23.99',
       '21.99', '15.0', '189.96', '139.92', '23.96', '26.99', '16.99',
       '7.49', '10.93', '19.98', '87.94', '69.99', '16.06', '13.37',
       '10.99', '79.99', '1.0', '0.5', '49.0', '771.71', '20.99', '99.99',
       '4.49', '9.95', '124.99', '10.0', '0.89', '1.29', '18.9', '4.0',
       '3.0', '2.0', '31.99', '119.99', '9.69', '299.99', '29.96', '9.98',
       '1.5', '8.98', '12.0', '9.0', '5.65', '74.99', '2.89', '449.0'],
      dtype=object)

Notamos que hay regitros con valores '--', asi que eliminamos las filas que contengan este valor.

In [19]:
df_function1 = df_function1.drop(df_function1[df_function1['price'] == '--'].index)

Corroboramos

In [20]:
df_function1.price.unique()

array(['9.99', '4.99', '19.99', 0.0, '6.99', '7.99', '2.99', '14.99',
       '0.0', '54.99', '29.99', '39.99', '24.99', '1.99', '11.99', '0.49',
       '0.98', '0.99', '59.99', '3.99', '2.49', '74.76', '5.99', '32.99',
       '12.99', '49.99', '13.98', '8.99', '13.99', '34.99', '12.89',
       '20.0', '14.95', '61.99', '15.99', '18.99', '17.99', '160.91',
       '59.95', '44.99', '3.49', '1.87', '23.99', '21.99', '15.0',
       '189.96', '139.92', '23.96', '26.99', '16.99', '7.49', '10.93',
       '19.98', '87.94', '69.99', '16.06', '13.37', '10.99', '79.99',
       '1.0', '0.5', '49.0', '771.71', '20.99', '99.99', '4.49', '9.95',
       '124.99', '10.0', '0.89', '1.29', '18.9', '4.0', '3.0', '2.0',
       '31.99', '119.99', '9.69', '299.99', '29.96', '9.98', '1.5',
       '8.98', '12.0', '9.0', '5.65', '74.99', '2.89', '449.0'],
      dtype=object)

Ahora modificamos el tipo de dato.

In [21]:
df_function1['price'] = df_function1['price'].astype(float)

Procedemos a agrupar por usuario sumando los precios de los items adquiridos.

In [22]:
df_function_1 = df_function1.groupby('user_id')['price'].sum().reset_index()

In [23]:
df_function_1.head()

,user_id,price
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92
3,-2SV-vuLB-Kg,437.49
4,-404PageNotFound-,1514.31


Ahora este DataFrame lo unimos con ***'users_2'*** para obtener la cantidad de items por usuario.

In [24]:
df_function_1 = df_function_1.merge(users_2, on='user_id', how = 'left')

In [25]:
df_function_1.head()

,user_id,price,items_count
0,--000--,402.77,58
1,--ace--,166.82,44
2,--ionex--,109.92,23
3,-2SV-vuLB-Kg,437.49,68
4,-404PageNotFound-,1514.31,149


Bien, ahora vamos a agrupar por usuario el DataFrame ***'reviews_2'*** para obtener la cantidad de items recomendados.

In [26]:
df_review = reviews_2.groupby('user_id')['recommend'].sum().reset_index()

In [27]:
df_review.head()

,user_id,recommend
0,--000--,1
1,--ace--,2
2,--ionex--,2
3,-2SV-vuLB-Kg,5
4,-Azsael-,1


Procedemos a unirlo con el DataFRame ***df_function_1***

In [28]:
df_function_1 = df_function_1.merge(df_review, on='user_id', how = 'left')

In [29]:
df_function_1.head()

,user_id,price,items_count,recommend
0,--000--,402.77,58,1.0
1,--ace--,166.82,44,2.0
2,--ionex--,109.92,23,2.0
3,-2SV-vuLB-Kg,437.49,68,5.0
4,-404PageNotFound-,1514.31,149,NaN


In [30]:
df_function_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70871 entries, 0 to 70870
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      70871 non-null  object 
 1   price        70871 non-null  float64
 2   items_count  70871 non-null  int64  
 3   recommend    22613 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.2+ MB


Vemos que hay valores nullos en la columnas *'recommend'*, asi que lo cambiaremos por 0

In [31]:
df_function_1['recommend'].fillna(0, inplace= True)

In [32]:
df_function_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70871 entries, 0 to 70870
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      70871 non-null  object 
 1   price        70871 non-null  float64
 2   items_count  70871 non-null  int64  
 3   recommend    70871 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.2+ MB


Ahora crearemos una columna con el valor del porcentaje de recomendacion por usuario.

In [33]:
df_function_1['percent_recommend'] = (df_function_1['recommend'] / df_function_1['items_count'] * 100).round(2)

In [34]:
df_function_1.head()

,user_id,price,items_count,recommend,percent_recommend
0,--000--,402.77,58,1.0,1.72
1,--ace--,166.82,44,2.0,4.55
2,--ionex--,109.92,23,2.0,8.70
3,-2SV-vuLB-Kg,437.49,68,5.0,7.35
4,-404PageNotFound-,1514.31,149,0.0,0.00


Renombramos las columnas para entender mejor el DataFrame.

In [35]:
df_function_1.rename(columns={'recommend': 'recommend_total','price': 'total_spent'}, inplace=True)

Guardamos el DataFrame en un archivo CSV para realizar la función de la API.

In [47]:
df_function_1.to_csv('./dataset/Endpoints/endpoint_userdata.csv', index= False)

In [1]:
# Prueba de funcion:

def userdata(user_id: str):
    df = df_function_1.loc[df_function_1['user_id'] == user_id]
    
    if not df.empty:
        user = df['user_id'].values[0]
        spent = df['total_spent'].values[0]
        percent_recommend = df['percent_recommend'].values[0]
        item_count = df['items_count'].values[0]
        return f'El usuario {user} gastó ${spent}, el cual recomendó un {percent_recommend}% y tiene {item_count} items'
    else:
        return 'No hay registros de ese usuario'


In [ ]:
userdata('76561197970982479')

# Función ***countreviews***

En esta función, se toma 2 parámetros lo cual son fechas, y se nos pide devolver la **cantidad de usuarios** que realizaron *reviews* entre las fechas dadas y el **porcentaje** de recomendacion.

Para esta función vamos a necesitar:
* user_id: ID del usuario
* posted: Fecha del posteo de la reseña
* recommend : Nos informa si el item es recomendado o no.

Sacamos las columnas necesarias

In [40]:
df_function_2 = reviews[['user_id','posted','recommend']]

Guardamos el DataFrame en un archivo CSV para realizar la función de la API.

In [46]:
df_function_2.to_csv('./dataset/Endpoints/endpoint_countreviews.csv', index= False)

In [12]:
def countreviews(desde:str, hasta:str):
    
    df = df_function_2[(df_function_2['posted'] >= desde) & (df_function_2['posted'] <= hasta)]

    if not df.empty:
        user_count = df['user_id'].nunique()
        recommend_count = df['recommend'].count()
        recommend_sum = df['recommend'].sum()
        recommend_percent = round(recommend_sum / recommend_count * 100, 2)
        return f'Desde {desde} hasta {hasta} realizaron reviews {user_count} usuarios, el cual solo el {recommend_percent}% recomendo juegos'
    else:
        return 'No hay registros entre esas fechas'

In [15]:
countreviews('2011-11-25', '2011-12-30')

'Desde 2011-11-25 hasta 2011-12-30 realizaron reviews 138 usuarios, el cual solo el 96.86% recomendo juegos'

# Función ***genre***

Esta función tomo un parametro que es el género y se nos pide devolver el puesto en el que se encuentra el género sobre el ranking de los mismos analizados bajo la columna *PlayTimeForever*.

Para esta función vamos a necesitar:
* genres: Género del juego
* playtime_total: Tiempo total jugado.
* ranking : Ranking segun el tiempo total de juego.

Vamos a seleccionar las columnas que necesitaremos

In [3]:
df_function_3 = items[['genres','playtime_forever']]

In [4]:
df_function_3.head()

,genres,playtime_forever
0,Action,6
1,Action,0
2,Action,7
3,Action,0
4,Action,0


Ahora agrupamos por género y sumamos la cantidad de tiempo de juego.

In [5]:
df_function_3 = df_function_3.groupby('genres')['playtime_forever'].sum().reset_index()

In [6]:
df_function_3.head()

,genres,playtime_forever
0,Action,3075100464
1,Adventure,898763639
2,Animation and Modeling,1994171
3,Audio Production,545201
4,Casual,249316634


Ordenamos de forma descendiente segun el tiempo total del juego.

In [7]:
df_function_3 = df_function_3.sort_values(by = 'playtime_forever', ascending = False)

In [8]:
df_function_3.head()

,genres,playtime_forever
0,Action,3075100464
9,Indie,1475473529
12,RPG,1027927921
1,Adventure,898763639
14,Simulation,855263068


Reiniciamos el índice del DataFrame

In [9]:
df_function_3.reset_index(drop= True, inplace= True)

Agregamos una columna que nos indique la posicion del ranking.

In [10]:
df_function_3['ranking'] = df_function_3['playtime_forever'].rank(ascending=False).astype(int)

In [11]:
df_function_3.head()

,genres,playtime_forever,ranking
0,Action,3075100464,1
1,Indie,1475473529,2
2,RPG,1027927921,3
3,Adventure,898763639,4
4,Simulation,855263068,5


Modificamos el nombre de la columna *'playtime_forever'*

In [12]:
df_function_3.rename(columns={'playtime_forever': 'playtime_total'}, inplace=True)

Guardamos el DataFrame en un archivo CSV

In [14]:
df_function_3.to_csv('./dataset/Endpoints/endpoint_genre.csv',index= False)

In [67]:
# Prueba de la funcón
def genre(genero: str):
    df = df_function_3[df_function_3['genres'] == genero]
    
    if not df.empty:
        genre = df.iloc[0, 0]
        ranking = df.iloc[0, 2]
        return f'El género {genre} se encuentra en el puesto número {ranking} del ranking'
    else:
        return 'No se encuentran registros de ese género'


In [66]:
genre('Simulation')

'El género Simulation se encuentra en el puesto número 5 del ranking'

# Función ***userforgenre***

Esta función toma un parámetro que es el género y se nos pide devolver el **TOP 5** de usuarios con mas horas de juego en el género dato, con su **URL(del usuario)** y **user_id**.

Para esta función vamos a necesitar:
* genres: Género del juego.
* user_id: ID del usuario.
* user_url: URL del usuario.
* playtime_total: Tiempo total jugado. 

Seleccionamos las columnas necesarias

In [15]:
items_3 = items[['user_id','genres','playtime_forever']]
users_3 = users[['user_id','user_url']]

Unimos los DataFrames para obtener el *'user_url'*

In [16]:
df_function_4 = items_3.merge(users_3, on= 'user_id', how= 'left')

In [17]:
df_function_4.head()

,user_id,genres,playtime_forever,user_url
0,76561197970982479,Action,6,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,Action,0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,Action,7,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,Action,0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,Action,0,http://steamcommunity.com/profiles/76561197970...


Agrupamos por las columnas *'genres'*, *'user_id'* y *'user_url'* y sumamos el tiempo total jugado.

In [18]:
df_function_4 = df_function_4.groupby(['genres','user_id','user_url'])['playtime_forever'].sum().reset_index()

In [19]:
df_function_4.head(10)

,genres,user_id,user_url,playtime_forever
0,Action,--000--,http://steamcommunity.com/id/--000--,139469
1,Action,--ace--,http://steamcommunity.com/id/--ace--,69325
2,Action,--ionex--,http://steamcommunity.com/id/--ionex--,38315
3,Action,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,42500
4,Action,-404PageNotFound-,http://steamcommunity.com/id/-404PageNotFound-,117423
5,Action,-AnimeIsMyThing-,http://steamcommunity.com/id/-AnimeIsMyThing-,166589
6,Action,-Azsael-,http://steamcommunity.com/id/-Azsael-,154523
7,Action,-Beave-,http://steamcommunity.com/id/-Beave-,34325
8,Action,-Encore-,http://steamcommunity.com/id/-Encore-,26357
9,Action,-GM-Dragon,http://steamcommunity.com/id/-GM-Dragon,25690


Ordenamos de manera descendente segun el tiempo total jugado y de manera ascendente segun el género.

In [20]:
df_function_4 = df_function_4.sort_values(by=['genres', 'playtime_forever'], ascending=[True, False])

In [21]:
df_function_4.head(10)

,genres,user_id,user_url,playtime_forever
50657,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307
64497,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580428
49457,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212
51237,Action,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742
44133,Action,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193
65224,Action,stopgovtcorruption,http://steamcommunity.com/id/stopgovtcorruption,1047411
43381,Action,Cow666,http://steamcommunity.com/id/Cow666,1031718
47167,Action,LapFucksTrax,http://steamcommunity.com/id/LapFucksTrax,885920
49143,Action,PiozZ,http://steamcommunity.com/id/PiozZ,874761
44213,Action,Drxoid,http://steamcommunity.com/id/Drxoid,806875


Reiniciamos el índice

In [22]:
df_function_4.reset_index(drop=True, inplace=True)

In [23]:
df_function_4.head(10)

,genres,user_id,user_url,playtime_forever
0,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307
1,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580428
2,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212
3,Action,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742
4,Action,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193
5,Action,stopgovtcorruption,http://steamcommunity.com/id/stopgovtcorruption,1047411
6,Action,Cow666,http://steamcommunity.com/id/Cow666,1031718
7,Action,LapFucksTrax,http://steamcommunity.com/id/LapFucksTrax,885920
8,Action,PiozZ,http://steamcommunity.com/id/PiozZ,874761
9,Action,Drxoid,http://steamcommunity.com/id/Drxoid,806875


Ahora creamos una columna que nos indica la posicion del ranking por género.

In [24]:
df_function_4['ranking'] = df_function_4.groupby('genres').cumcount() + 1

In [25]:
df_function_4.head(10)

,genres,user_id,user_url,playtime_forever,ranking
0,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307,1
1,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580428,2
2,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212,3
3,Action,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742,4
4,Action,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193,5
5,Action,stopgovtcorruption,http://steamcommunity.com/id/stopgovtcorruption,1047411,6
6,Action,Cow666,http://steamcommunity.com/id/Cow666,1031718,7
7,Action,LapFucksTrax,http://steamcommunity.com/id/LapFucksTrax,885920,8
8,Action,PiozZ,http://steamcommunity.com/id/PiozZ,874761,9
9,Action,Drxoid,http://steamcommunity.com/id/Drxoid,806875,10


Agrupamos el DataFrame por género y tomamos solo los primeros 5 registros por cada género

In [26]:
df_function_4['top5_users'] = df_function_4.groupby('genres')['user_id'].head(5)

In [27]:
df_function_4.head(10)

,genres,user_id,user_url,playtime_forever,ranking,top5_users
0,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307,1,Sp3ctre
1,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580428,2,shinomegami
2,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212,3,REBAS_AS_F-T
3,Action,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742,4,Terminally-Chill
4,Action,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193,5,DownSyndromeKid
5,Action,stopgovtcorruption,http://steamcommunity.com/id/stopgovtcorruption,1047411,6,NaN
6,Action,Cow666,http://steamcommunity.com/id/Cow666,1031718,7,NaN
7,Action,LapFucksTrax,http://steamcommunity.com/id/LapFucksTrax,885920,8,NaN
8,Action,PiozZ,http://steamcommunity.com/id/PiozZ,874761,9,NaN
9,Action,Drxoid,http://steamcommunity.com/id/Drxoid,806875,10,NaN


Eliminamos los registros vacios de la columna *'top5_users'*

In [28]:
df_function_4.dropna(subset='top5_users',inplace=True)

Reiniciamos el índice

In [30]:
df_function_4.reset_index(drop=True, inplace=True)

In [31]:
df_function_4.head(10)

,genres,user_id,user_url,playtime_forever,ranking,top5_users
0,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307,1,Sp3ctre
1,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580428,2,shinomegami
2,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212,3,REBAS_AS_F-T
3,Action,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742,4,Terminally-Chill
4,Action,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193,5,DownSyndromeKid
5,Adventure,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,2191551,1,REBAS_AS_F-T
6,Adventure,shinomegami,http://steamcommunity.com/id/shinomegami,767160,2,shinomegami
7,Adventure,Evilutional,http://steamcommunity.com/id/Evilutional,662103,3,Evilutional
8,Adventure,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,654378,4,Sp3ctre
9,Adventure,tobscene,http://steamcommunity.com/id/tobscene,640783,5,tobscene


Eliminamos la columna *'playtime_forever'*

In [33]:
df_function_4.drop(columns='playtime_forever',inplace= True)

Guardamos el DataFrame en un archivo CSV

In [35]:
df_function_4.to_csv('./dataset/Endpoints/endpoint_userforgenre.csv', index= False)

In [36]:
def userforgenre(genero: str):
    if genero in df_function_4['genres'].unique():
        df = df_function_4[df_function_4['genres'] == genero]

        ranking = [f'Nro {i + 1}' for i in range(len(df))]

        users_info = [
            f'Usuario: {user_id} | URL del usuario: {url_user}'
            for user_id, url_user in zip(df['user_id'], df['user_url'])
        ]

        return dict(zip(ranking, users_info))
    else:
        return 'No hay registros con ese género'


In [37]:
userforgenre('Action')

{'Nro 1': 'Usuario: Sp3ctre | URL del usuario: http://steamcommunity.com/id/Sp3ctre',
 'Nro 2': 'Usuario: shinomegami | URL del usuario: http://steamcommunity.com/id/shinomegami',
 'Nro 3': 'Usuario: REBAS_AS_F-T | URL del usuario: http://steamcommunity.com/id/REBAS_AS_F-T',
 'Nro 4': 'Usuario: Terminally-Chill | URL del usuario: http://steamcommunity.com/id/Terminally-Chill',
 'Nro 5': 'Usuario: DownSyndromeKid | URL del usuario: http://steamcommunity.com/id/DownSyndromeKid'}

# Función ***developer***

Esta función toma un parámetro que es el nombre de la empresa Desarrolladora, y se nos pide devolver el **porcentaje** de contenido *FREE* por año segun la empresa.

Para esta función vamos a necesitar:
* developer: Empresa desarrolladora del juego.
* year: Año del lanzamiento del juego.
* items_count: Cantidad de juegos lanzados.
* items_count_free: Cantidad de juegos FREE lanzados. 
* free_percentage: Porcentaje de juegos FREE lanzados.

Seleccionamos las columnas necesarias

In [86]:
df_function_5 = steam_games[['developer','release_year','id']]

In [87]:
df_function_5.head()

,developer,release_year,id
0,Kotoshiro,2018.0,761140.0
1,Secret Level SRL,2018.0,643980.0
2,Poolians.com,2017.0,670290.0
3,彼岸领域,2017.0,767400.0
4,NaN,NaN,773570.0


In [88]:
u.valores_nulos(df_function_5)

La columna developer tiene un 10.3% de valores nulos.
La columna release_year tiene un 6.6% de valores nulos.
La columna id tiene un 0.0% de valores nulos.


Borramos los registros vacios

In [89]:
df_function_5.dropna(inplace=True)

Reiniciamos el índice

In [90]:
df_function_5.reset_index(drop= True, inplace= True)

Agrupamos por *'developer'* y *'release_year'* y contamos los id unicos.

In [91]:
df_function_5 = df_function_5.groupby(['developer','release_year'])['id'].nunique().reset_index()

In [92]:
df_function_5

,developer,release_year,id
0,+7 Software,2016.0,1
1,"+Mpact Games, LLC.",2017.0,1
2,.M.Y.W.,2016.0,1
3,.ez Games,2017.0,1
4,07th Expansion,2015.0,2
...,...,...,...
14977,萌石游戏,2017.0,1
14978,高考恋爱委员会,2015.0,1
14979,"高考恋爱委员会,Days",2015.0,1
14980,"高考恋爱委员会,橘子班",2015.0,1


Renombramos las columnas para poder entender mejor el DataFrame

In [93]:
df_function_5.rename(columns={'id': 'items_count','release_year': 'year'}, inplace=True)

Cambiamos el tipo de dato de las columnas *year* y *items_count*

In [94]:
df_function_5[['items_count','year']] = df_function_5[['items_count','year']].astype(int)

Ahora creamos un nuevo DataFrame donde los items sean FREE, o sea, el precio sea igual a cero.

In [49]:
developer_free = steam_games[['developer', 'id', 'release_year', 'price']][steam_games['price'] == '0.0']

In [51]:
developer_free.head()

,developer,id,release_year,price
1,Secret Level SRL,643980.0,2018.0,0.0
2,Poolians.com,670290.0,2017.0,0.0
11,NaN,724910.0,NaN,0.0
60,Unknown Worlds Entertainment,4900.0,2006.0,0.0
190,Sandstorm Productions,1230.0,2008.0,0.0


Bien, borramos la columna *'price'*

In [52]:
developer_free.drop(columns= 'price', inplace= True)

Hacemos la misma agrupación que el otro DataFrame

In [53]:
developer_free = developer_free.groupby(['developer', 'release_year'])['id'].nunique().reset_index()

Y ahora cambiamos los nombres de las columnas

In [54]:
developer_free.rename(columns={'id': 'items_count_free','release_year': 'year'}, inplace=True)

In [55]:
developer_free.head()

,developer,year,items_count_free
0,14 Hours Productions,2017.0,1
1,2-Volt Games,2017.0,1
2,"2Chance Projects,IIchan Eroge Team",2015.0,1
3,3D Realms,1996.0,1
4,4 I Lab,2016.0,1


Cambiamos el tipo de dato de la columna *'year'*

In [56]:
developer_free['year'] = developer_free['year'].astype(int)

Procedemos a unir ambos DataFrames 

In [95]:
df_function_5 = df_function_5.merge(developer_free, on=['developer', 'year'], how='left')

In [96]:
df_function_5.head()

,developer,year,items_count,items_count_free
0,+7 Software,2016,1,NaN
1,"+Mpact Games, LLC.",2017,1,NaN
2,.M.Y.W.,2016,1,NaN
3,.ez Games,2017,1,NaN
4,07th Expansion,2015,2,NaN


Vemos que hay registros vacios en la columna ***items_count_free***, y nos da a entender de que hay ausencia de contenido FREE, asi que lo reemplazaremos por un cero.

In [97]:
df_function_5.items_count_free.fillna(0, inplace= True)

Ahora crearemos una nueva columna que nos indicara el porcentaje de contenido FREE por desarrollador y año

In [98]:
df_function_5['free_percentage'] = (df_function_5["items_count_free"] / df_function_5["items_count"] * 100).round(2)

In [99]:
df_function_5 

,developer,year,items_count,items_count_free,free_percentage
0,+7 Software,2016,1,0.0,0.0
1,"+Mpact Games, LLC.",2017,1,0.0,0.0
2,.M.Y.W.,2016,1,0.0,0.0
3,.ez Games,2017,1,0.0,0.0
4,07th Expansion,2015,2,0.0,0.0
...,...,...,...,...,...
14977,萌石游戏,2017,1,0.0,0.0
14978,高考恋爱委员会,2015,1,1.0,100.0
14979,"高考恋爱委员会,Days",2015,1,0.0,0.0
14980,"高考恋爱委员会,橘子班",2015,1,0.0,0.0


Guardamos el DataFrame en un archivo CSV

In [65]:
df_function_5.to_csv('./dataset/Endpoints/endpoint_developer.csv', index= False)

In [105]:
def developer(desarrollador: str):
    if desarrollador in df_function_5['developer'].unique():
        df = df_function_5[df_function_5['developer'] == desarrollador]

        mi_dict = {
            year: f'Cantidad de Items: {items_count} || Contenido FREE: {free_percent:}%'
            for year, items_count, free_percent in zip(df['year'], df['items_count'], df['free_percentage'])
        }

        return mi_dict
    else:
        return 'Desarrollador no encontrado en los datos'


In [106]:
developer('Activision')

{1993: 'Cantidad de Items: 1 || Contenido FREE: 0.0%',
 1996: 'Cantidad de Items: 1 || Contenido FREE: 0.0%',
 1997: 'Cantidad de Items: 1 || Contenido FREE: 0.0%',
 2000: 'Cantidad de Items: 1 || Contenido FREE: 0.0%'}

# Función ***sentiment_analysis***

Esta función toma un parámetro que es el año del posteo de la reseña y devuelve una lista con la cantidad de reseñas categorizadas con un análisis de sentimiento.


Para esta función vamos a necesitar:
* user_id: ID del usuario.
* item_id: ID del juego.
* sentiment_analysis: Categoria del análisis de sentimiento de la reseña.
* year_posted: Añio del posteo de la reseña. 

Seleccionamos las columnas necesarias

In [5]:
df_function_6 = reviews[['user_id','item_id','posted','sentiment_analysis']]

In [6]:
df_function_6.head()

,user_id,item_id,posted,sentiment_analysis
0,76561197970982479,1250,2011-11-05,2
1,76561197970982479,22200,2011-07-15,2
2,76561197970982479,43110,2011-04-21,2
3,js41637,251610,2014-06-24,2
4,js41637,227300,2013-09-08,2


Verificamos los valores nulos

In [7]:
u.valores_nulos(df_function_6)

La columna user_id tiene un 0.0% de valores nulos.
La columna item_id tiene un 0.0% de valores nulos.
La columna posted tiene un 17.0% de valores nulos.
La columna sentiment_analysis tiene un 0.0% de valores nulos.


Eliminamos esos registros con valores nulos

In [8]:
df_function_6.dropna(inplace= True)

In [9]:
df_function_6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48498 entries, 0 to 58402
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             48498 non-null  object
 1   item_id             48498 non-null  int64 
 2   posted              48498 non-null  object
 3   sentiment_analysis  48498 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.9+ MB


Cambiamos el tipo de dato de la columnas *'posted'*

In [10]:
df_function_6['posted'] = pd.to_datetime(df_function_6['posted'], format='%Y-%m-%d')

Agregamos una columna con el año del posteo para facilitar el analisis

In [11]:
df_function_6['year_posted'] = df_function_6['posted'].dt.year

In [12]:
df_function_6.head()

,user_id,item_id,posted,sentiment_analysis,year_posted
0,76561197970982479,1250,2011-11-05,2,2011
1,76561197970982479,22200,2011-07-15,2,2011
2,76561197970982479,43110,2011-04-21,2,2011
3,js41637,251610,2014-06-24,2,2014
4,js41637,227300,2013-09-08,2,2013


Eliminamos la columna *'posted'*

In [13]:
df_function_6.drop(columns='posted',inplace= True)

Guardamos el DataFrame en un archivo CSV

In [15]:
df_function_6.to_csv('./dataset/Endpoints/endpoint_sentiment_analysis.csv', index= False)

In [124]:
def sentiment_analysis(year: int,):

    if year in df_function_6['year_posted'].unique():
        df = df_function_6[df_function_6['year_posted'] == year]

        sentiment_counts = df['sentiment_analysis'].value_counts().to_dict()

        result= {
            'Negativo': sentiment_counts.get(0, 0),
            'Neutral': sentiment_counts.get(1, 0),
            'Positivo': sentiment_counts.get(2, 0)
        }
        return result
    else:
        return 'No hay registros con ese año'


In [125]:
sentiment_analysis(2015)

{'Negativo': 3323, 'Neutral': 3853, 'Positivo': 10978}

# Modelo de Recomendación

Para el Modelo de Recomendación, vamos a usar el enfoque **Collaborative Filtering** el cual necesitamos información de Rating.
Como en los datos no tenemos informacion de Rating de los usuarios por los ítems, debemos crear uno, ya que esto nos ayudara a ver:
- Usuarios con preferencias similares.
- Características en los items que influencian los ratings de los usuarios.

### Lectura de los datasets a utilizar

In [154]:
steam_games = pd.read_csv('./dataset/Modificados/steam_games.csv')
reviews = pd.read_csv('./dataset/Modificados/reviews_sentiment_analysis.csv')

### Creacion del Rating

Usamos el dataset *reviews* para crear el rating.
En este dataset tenemos la informacion de si un usuario recomendo un juego y la clasificacion de la reseña por análisis de sentimiento. Asi que vamos a generar un rating con rango del 1 al 5 para estos casos:
- Rating 1: Si el usuario no recomienda el juego.
- Rating 2: Si el usuario no recomienda el juego y la clasificacion del análisis de sentimiento es Neutral (1)
- Rating 3: Si el usuario recomienda el juego y la clasificacion del análisis de sentimiento es Neutral (1)
- Rating 4: Si el usuario no recomienda el juego y la clasificacion del análisis de sentimiento es Positiva (2)
- Rating 5: Si el usuario recomienda el juego y la clasificacion del análisis de sentimiento es Positiva (2)

Elegimos los datos que necesitamos

In [155]:
reviews_ = reviews[['user_id','item_id','recommend','sentiment_analysis']]

Creamos una función para los valores de la columna *Rating*

In [156]:
def calcula_rating(row):
    """
    Cálculo del rating.

    Esta función calcula el rating del 1 al 5
    en base a la columna ``recommend`` y ``sentiment_analysis``
    del Dataframe ``reviews``.

    Parameters
    ----------
    row: dict
        Un diccionario que contiene las siguientes claves:
            - "sentiment_analysis" (int): La clasificación del análisis de sentimientos (0, 1 o 2).
            - "recommend" (bool): Indica si un item fue recomendado o no.

    Returns
    -------
    int
        El número del rating entre el 1 y 5.
    """

    rating_map = {
        (0, False): 1,
        (0,True): 1,
        (1, False): 2,
        (1, True): 3,
        (2, False): 4,
        (2, True): 5,
    }
    return rating_map.get((row["sentiment_analysis"], row["recommend"]), None)


Aplicamos la función

In [158]:
reviews_['rating'] = reviews_.apply(calcula_rating, axis= 1)

In [160]:
reviews_.head()

,user_id,item_id,recommend,sentiment_analysis,rating
0,76561197970982479,1250,True,2,5
1,76561197970982479,22200,True,2,5
2,76561197970982479,43110,True,2,5
3,js41637,251610,True,2,5
4,js41637,227300,True,2,5


Eliminamos las columnas que seran irrelevantes para el modelo

In [161]:
reviews_.drop(columns=['recommend','sentiment_analysis'], inplace= True)

Ahora vamos a extraer el nombre de los ítems.

In [163]:
steam_ = steam_games[['id','app_name']]

Unimos ambos DataFrames

In [166]:
df_model = reviews_.merge(steam_, left_on='item_id', right_on='id', how='left')

Eliminamos la columna *'id'*, ya que tenemos la misma informacion en la columna *'item_id'*

In [169]:
df_model.drop(columns= 'id',inplace= True)

In [170]:
df_model.head()

,user_id,item_id,rating,app_name
0,76561197970982479,1250,5,Killing Floor
1,76561197970982479,22200,5,Zeno Clash
2,76561197970982479,43110,5,NaN
3,js41637,251610,5,NaN
4,js41637,227300,5,Euro Truck Simulator 2


Ordenamos las columnas

In [171]:
df_model = df_model[['user_id','item_id','app_name','rating']]

Guardamos el DataFrame en un archivo CSV

In [172]:
df_model.to_csv('./dataset/Endpoints/model_recommend.csv', encoding= 'utf-8', index= False)